In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os.path
from fastai import *

In [2]:
from PIL import Image
import pandas as pd 
import numpy as np 

In [3]:
from fastai.text import *
PATH=Path('data')

%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [4]:
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")

In [5]:
train = train.head(5000)

In [6]:
test = test.head(5000)

In [7]:
import fastText as ft

In [8]:
ru_vecs = ft.load_model('data/wiki.ru.bin')

In [9]:
def get_vecs(lang, ft_vecs):
    vecd = {w:ft_vecs.get_word_vector(w) for w in ft_vecs.get_words()}
    pickle.dump(vecd, open(PATH/f'wiki.{lang}.pkl', 'wb'))
    return vecd

In [10]:
ru_vecd = get_vecs('ru', ru_vecs)

In [11]:
ru_vecd = pickle.load(open(PATH/f'wiki.ru.pkl', 'rb'))

In [12]:
ft_words = ru_vecs.get_words(include_freq=True)
ft_word_dict = {k:v for k,v in zip(*ft_words)}
ft_words = sorted(ft_word_dict.keys(), key=lambda x: ft_word_dict[x])
len(ft_words)

1888423

In [13]:
dim_ru_vec = len(ru_vecd[','])
dim_ru_vec

300

In [14]:
ru_vecs = np.stack(list(ru_vecd.values()))
ru_vecs.mean(), ru_vecs.std()

(0.003615052, 0.29542154)

In [15]:
len(test), len(train)

(5000, 5000)

Turning CSV into lines

In [16]:
import re
re.sub("[()]", " ", "foo(a)")

'foo a '

In [17]:
#df = pd.DataFrame(columns=['text', "probability"])
#dict = {} 
text = []
probability = []
for i, r in train.iterrows():
    t = f'{r.region} {r.city} {r.parent_category_name} {r.category_name} {r.param_1} {r.param_2} {r.param_3} {r.title} {r.description} {r.price} {r.item_seq_number} {r.user_type} {"yes" if r.image else "no"}'
    #t = f'{r.description}'
    p = r.deal_probability
    text.append(re.sub("[()]", " ", t))
    probability.append(p)
    #df.loc[len(df)] = [text, probability]
    if i % 100000 == 0: print(i)
    
#train_processed = df

0


In [18]:
pickle.dump(text, open('data/text.pkl', 'wb'))
pickle.dump(probability, open('data/probability.pkl', 'wb'))

In [19]:
text = pickle.load(open('data/text.pkl', 'rb'))
probability = pickle.load(open('data/probability.pkl', 'rb'))

In [20]:
%time text_tok = Tokenizer.proc_all_mp(partition_by_cores(text))

CPU times: user 76 ms, sys: 368 ms, total: 444 ms
Wall time: 4.41 s


In [21]:
text_tok[0]

['свердловская',
 'область',
 'екатеринбург',
 'личные',
 'вещи',
 'товары',
 'для',
 'детей',
 'и',
 'игрушки',
 'постельные',
 'принадлежности',
 'nan',
 'nan',
 'кокоби',
 'кокон',
 'для',
 'сна',
 'кокон',
 'для',
 'сна',
 'малыша,пользовались',
 'меньше',
 'месяца.цвет',
 'серый',
 '400.0',
 '2',
 'private',
 'yes']

In [22]:
np.percentile([len(o) for o in text_tok], 90), np.percentile([len(o) for o in text_tok], 90)

(101.0, 101.0)

In [23]:
keep = np.array([len(o) < 101 for o in text_tok])

In [24]:
text_tok = np.array(text_tok)[keep]
probability = np.array(probability)[keep]

In [25]:
pickle.dump(text_tok, open('data/text_tok.pkl', 'wb'))
pickle.dump(probability, open('data/probability_k.pkl', 'wb'))


In [26]:
text_tok=pickle.load(open('data/text_tok.pkl', 'rb'))
probability=pickle.load(open('data/probability_k.pkl', 'rb'))

In [27]:
len(probability), len(text_tok)

(4499, 4499)

In [28]:
def toks2ids(tok, pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(40000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    itos.insert(4, 'nan')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([stoi[o] for o in p] + [2]) for p in tok])
    np.save(f'data/tmp_{pre}_ids.npy', ids)
    pickle.dump(itos, open(f'data/tmp_{pre}_itos.pkl', 'wb'))
    return ids, itos, stoi

In [29]:
text_ids, text_itos, text_stoi = toks2ids(text_tok, 'text')

In [30]:
def load_ids(pre):
    ids = np.load(f'data/tmp_{pre}_ids.npy')
    itos = pickle.load(open(f'data/tmp_{pre}_itos.pkl', 'rb'))
    stoi = collections.defaultdict(lambda: 3, {v:k for k, v in enumerate(itos)})
    return ids, itos, stoi

In [31]:
text_ids, text_itos, text_stoi = load_ids('text')

In [32]:
len(probability), len(text_ids)

(4499, 4499)

Training and validation data set

In [33]:
np.random.seed(42)
trn_keep = np.random.rand(len(text_ids)) > 0.1
trn = text_ids[trn_keep]
trn_y = probability[trn_keep]
val = text_ids[~trn_keep]
val_y = probability[~trn_keep]
len(trn), len(val)

(4017, 482)

In [158]:
trn

array([ list([76, 12, 120, 17, 16, 51, 11, 79, 7, 61, 615, 599, 6, 6, 8026, 2795, 11, 3503, 2795, 11, 3503, 8027, 1038, 8028, 1180, 154, 29, 9, 8, 2]),
       list([93, 12, 157, 11, 35, 7, 38, 82, 7, 111, 109, 6, 6, 2796, 11, 694, 2796, 11, 694, 5, 138, 8029, 23, 8030, 226, 247, 9, 8, 2]),
       list([90, 12, 174, 39, 44, 282, 7, 232, 232, 5, 21, 1181, 7, 1586, 18, 1587, 1783, 6, 6, 1784, 8031, 14, 58, 52, 5, 1588, 8032, 23, 1586, 1587, 5, 21, 1039, 28, 573, 8033, 5, 574, 311, 2797, 1785, 13, 20, 112, 396, 113, 9, 8, 2]),
       ...,
       list([150, 27, 254, 100, 7, 67, 245, 7, 67, 711, 7, 741, 6, 6, 23144, 1157, 23145, 23146, 8023, 14, 64, 23147, 85, 23148, 13, 20, 11, 748, 1445, 23149, 300, 23150, 13, 20, 7769, 810, 23151, 87, 23152, 23153, 23154, 7, 23155, 138, 23156, 23157, 19, 2024, 23158, 23159, 11, 3496, 7, 23160, 8023, 249, 7991, 512, 182, 23161, 23162, 778, 23163, 9, 8, 2]),
       list([257, 313, 17, 16, 10, 5, 15, 5, 24, 33, 10, 46, 10, 129, 115, 108, 119, 119, 5, 433, 44

In [157]:
trn_y

array([ 0.12789,  0.     ,  0.43177, ...,  0.08065,  0.     ,  0.     ])

In [34]:
class Seq2SeqDataset(Dataset):
    def __init__(self, x, y): self.x, self.y = x, y
    def __getitem__(self, idx): return A(self.x[idx], [int(self.y[idx]*1000)])
    def __len__(self): return len(self.x)

In [35]:
int(0.2312)

0

In [36]:
#trn_y = trn.iloc[:,-1]
#val_y = val.iloc[:,-1]

In [37]:
#trn = trn.iloc[:, :-1]
#val = val.iloc[:, :-1]

In [38]:
trn_ds = Seq2SeqDataset(trn, trn_y)
val_ds = Seq2SeqDataset(val, val_y)

In [233]:
batch_size=125

In [234]:
trn_samp = SortishSampler(trn, key=lambda x: len(trn[x]), bs=batch_size)
val_samp = SortSampler(val, key=lambda x: len(val[x]))

In [235]:
trn_dl = DataLoader(trn_ds, batch_size, transpose = True, transpose_y=True, num_workers=1, pad_idx=1, pre_pad=False,
                   sampler=trn_samp)
val_dl = DataLoader(val_ds, int(batch_size*1.6), transpose=True, transpose_y=True, num_workers=1, pad_idx = 1,
                pre_pad = False, sampler=val_samp)
md = ModelData(PATH, trn_dl, val_dl)

In [236]:
def create_emb(vecs, itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx = 1)
    wgts = emb.weight.data # jeremy: this is a tensor
    miss = []
    for i, w in enumerate(itos):
        try: wgts[i] = torch.from_numpy(vecs[w] * 3) # jeremy: our stddev is 0.3, so mul by 3 to get to 1.0
        except: miss.append(w)
    print(len(miss), miss[5:10])
    return emb

In [237]:
num_hidden_features, num_layers = 16, 2

In [238]:
class Seq2SeqRNN(nn.Module):
    def __init__(self, vecs_enc, itos_enc, em_sz_enc, vecs_dec, itos_dec, em_sz_dec, num_hidden_features, out_sequence_length, num_layers=2):
        super().__init__()
        self.num_layers, self.num_hidden_features, self.out_sequence_length = num_layers, num_hidden_features, out_sequence_length
        self.emb_enc = create_emb(vecs_enc, itos_enc, em_sz_enc)
        self.emb_enc_drop = nn.Dropout(0.15)
        self.gru_enc = nn.GRU(em_sz_enc, num_hidden_features, num_layers=num_layers, dropout=0.25)
        self.out_enc = nn.Linear(num_hidden_features, em_sz_dec, bias=False)
        
        self.emb_dec = create_emb(vecs_dec, itos_dec, em_sz_dec)
        self.gru_dec = nn.GRU(em_sz_dec, em_sz_dec, num_layers=num_layers, dropout=0.1)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(em_sz_dec, len(itos_dec))
        self.out.weight.data = self.emb_dec.weight.data
    
    def forward(self, inp):
        sequence_length, batch_size = inp.size()
        h = self.initHidden(batch_size)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, h = self.gru_enc(emb, h)
        h = self.out_enc(h)
        
        dec_inp = V(torch.zeros(batch_size).long())
        res = []
        
        for i in range(self.out_sequence_length):
            # dec_inp: the previous word we translated
            emb = self.emb_dec(dec_inp).unsqueeze(0) # treat this as a sequence of length=1
            outp, h = self.gru_dec(emb, h)
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            dec_inp = V(outp.data.max(1)[1])
            if (dec_inp==1).all(): break
        return torch.stack(res)
    
    def initHidden(self, batch_size): return V(torch.zeros(self.num_layers, batch_size, self.num_hidden_features))
      


In [239]:
def seq2seq_loss(input, target):
    sequence_length, batch_size = target.size()
    sequence_length_in, batch_size_in, nc = input.size()
    # add some padding, because our sequence length might be different 
    if sequence_length > sequence_length_in: input = F.pad(input, (0, 0, 0, 0, 0, sequence_length-sequence_length_in))
    input = input[:sequence_length]
    return F.cross_entropy(input.view(-1, nc), target.view(-1)) 
    #return F.mse_loss(input.view(-1, nc), target.view(-1)) 

In [240]:
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [241]:
dim_ru_vec, len(text_itos)

(300, 23166)

In [242]:
rnn = Seq2SeqRNN(ru_vecd, text_itos, dim_ru_vec, ru_vecd, probability, 1, num_hidden_features, 101)
learn = RNN_Learner(md, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.crit = seq2seq_loss

10080 ['t_up', '2', '4', '1', 'tk_rep']
4499 [0.80323, 0.0, 0.80323, 0.0, 0.0]


In [243]:
#learn.lr_find()
#learn.sched.plot()

In [244]:
learning_rate = 10e-2

In [245]:
#??learn.fit

In [246]:
learn.fit(learning_rate, 1, cycle_len=12, use_clr=(20, 10))

epoch      trn_loss   val_loss                            
    0      6.927715   3.566764  
    1      4.642829   3.06099                             
    2      3.64458    2.650518                            
    3      3.106591   2.632878                            
    4      2.779962   2.595338                            
    5      2.652441   2.601975                            
    6      2.53254    2.566906                            
    7      2.472813   2.550843                            
    8      2.414332   2.532799                            
    9      2.406546   2.547247                            
    10     2.353711   2.546311                            
    11     2.33129    2.548141                            



[array([ 2.54814])]

In [ ]:
learn.save("initial.m")

In [ ]:
learn.load("initial.m")

Test

In [ ]:
x, y = next(iter(val_dl))
probs = learn.model(V(x))
preds = to_np(probs.max(2)[1])


In [ ]:
str(10)